In [ ]:
import pandas as pd
import numpy as np
import re

# Parser

For data scraping has been used BeautifulSoup [parser](https://github.com/morn1n9st4r/LiveLib_DS_Project/blob/master/BookParser.py).

In [ ]:
df = pd.read_csv('livelib_books.csv')

In [ ]:
df.head()

,BookTitle,Author,ISBN,EditionYear,Pages,Size,CoverType,Language,CopiesIssued,AgeRestrictions,Genres,TranslatorName,Rating,HaveRead,Planned,Reviews,Quotes,Series,Edition
0,Поль Скаррон. Комический роман,Поль Скаррон,None,1934,None,None,None,Русский,None,None,['Классическая литература'],['Н. Кравцовстр. 5-645'],3,1,17,1,None,Academia,None
1,Пороховой маг. Книга 3. Кровавая осень,Брайан Макклеллан,['978-5-389-10393-1'],2016,544,None,Твердый,Русский,None,16,"['Зарубежное фэнтези', 'Героическая фантастика']",['С. Удалин'],"4,3",240,305,11,42,Звезды новой фэнтези,"Азбука, Азбука-Аттикус"
2,В сердце моря,Натаниэль Филбрик,['978-5-699-78826-2'],2015,384,84x108/32,Твердый,Русский,5 000,18,"['Современная зарубежная литература', 'Морские...",None,"4,2",185,626,14,11,"Книга, покорившая мир",Эксмо
3,Долго и счастливо,Карина Пьянкова,['978-5-9922-2065-0'],2015,313,84х108/32,None,Русский,4 000,16,['Русское фэнтези'],None,"3,8",97,103,2,3,Романтическая фантастика,Альфа-книга
4,Собрание сочинений в двадцати шести томах. Том...,Эмиль Золя,None,1964,664,84x108/32,твёрдая,Русский,297000,None,"['Зарубежная классика', 'Классическая проза', ...","['М. Роммстр. 7Человек-зверь — Эмиль Золя', ' ...","4,1",24,17,95,236,Художественная литература,None


# Arrays

Some of features are array-like so we need to remove square-like brackets from them.

In [ ]:
df['Genres'] = df['Genres'].str.replace(r'[', '', regex=True).astype('str')
df['Genres'] = df['Genres'].str.replace(r']', '', regex=True).astype('str')
df['Genres'] = df['Genres'].str.replace(r"'", '', regex=True).astype('str')

df['TranslatorName'] = df['TranslatorName'].str.replace(r'[', '', regex=True).astype('str')
df['TranslatorName'] = df['TranslatorName'].str.replace(r']', '', regex=True).astype('str')
df['TranslatorName'] = df['TranslatorName'].str.replace(r"'", '', regex=True).astype('str')

df['ISBN'] = df['ISBN'].str.replace(r'[', '', regex=True).astype('str')
df['ISBN'] = df['ISBN'].str.replace(r']', '', regex=True).astype('str')
df['ISBN'] = df['ISBN'].str.replace(r"'", '', regex=True).astype('str')

df['CopiesIssued'] = df['CopiesIssued'].str.replace(r" ", '', regex=True).astype('str')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  from ipykernel import kernelapp as app


# Planned, Rewies and Quotes

Most of users are not writing reviews and only cecking mark that they have read the book.

Replace nans with 0.

In [ ]:
df = df.replace(to_replace='None', value=np.nan)

In [ ]:
df['Planned'] = df['Planned'].fillna(0)
df['Reviews'] = df['Reviews'].fillna(0)
df['Quotes'] = df['Quotes'].fillna(0)

In [ ]:
df.isnull().sum()

BookTitle              0
Author               164
ISBN                4803
EditionYear         2067
Pages               9645
Size               17429
CoverType          13635
Language            2456
CopiesIssued       17545
AgeRestrictions    15618
Genres              1686
TranslatorName     16355
Rating                 0
HaveRead               0
Planned                0
Reviews                0
Quotes                 0
Series              1634
Edition             7968
dtype: int64

# ISBN - international standart book identifier


Some books have fer ISBN numbers. This occurs because parts of books have been released in different time.

Leave only one ISBN number.

In [ ]:
df["ISBN"] = df["ISBN"].str.replace(r', .+', '')

In [ ]:
df.loc[df['ISBN']!=np.nan]['ISBN'].is_unique

False

ISBN is not unique.

This means that some rows are either dublicates or LiveLib ave few entries for one book.

Group all books by ISBN column and then take only first unique row to avoid repeats.

In [ ]:
df_grouped = df.groupby("ISBN").first().reset_index()
df_grouped.head()

,ISBN,BookTitle,Author,EditionYear,Pages,Size,CoverType,Language,CopiesIssued,AgeRestrictions,Genres,TranslatorName,Rating,HaveRead,Planned,Reviews,Quotes,Series,Edition
0,0-000-00000-0,Опоздавшие к лету: Колдун. Мост Ватерлоо. Аттр...,Андрей Лазарчук,1990,276,76x90/32 (109x177 мм),твёрдая,Русский,45000,16,Отечественная фантастика,None,4,81,132,18,44,"Новая фантастика, вып.IV",Латвийский детский фонд
1,0-06-017012-3,A Suitable Boy,Vikram Seth,1993,None,None,None,Английский,None,None,Современная зарубежная литература,None,"4,2",21,349,1,0,HarperCollins,None
2,0-06-052970-9,Everything is illuminated,Jonathan Safran Foer,2002,None,None,None,Английский,None,16,Современная зарубежная литература,None,"3,9",4230,4798,262,530,Houghton Mifflin,None
3,0-06-113937-8,Coraline,Neil Gaiman,2006,None,None,None,Английский,None,12,"Зарубежные детские книги, Зарубежное фэнтези, ...",None,"4,2",19375,8158,920,434,Harper Perennial,None
4,0-091-74827-5,Фатерланд,Роберт Харрис,2000,384,None,None,Русский,None,None,"Триллеры, Политические детективы, Историческая...",Владимир Михайлов,"4,2",496,858,39,71,Торнтон и Сагден,None


In [ ]:
df_grouped.loc[df['ISBN']!=np.nan]['ISBN'].is_unique

True

In [ ]:
df.shape[0] - df_grouped.shape[0]

5176

# Inspect ISBN with None values

Let's look which books with no ISBN we removed.

In [ ]:
df_isbn_nans = df.loc[df["ISBN"].isnull()]
df_isbn_nans

,BookTitle,Author,ISBN,EditionYear,Pages,Size,CoverType,Language,CopiesIssued,AgeRestrictions,Genres,TranslatorName,Rating,HaveRead,Planned,Reviews,Quotes,Series,Edition
0,Поль Скаррон. Комический роман,Поль Скаррон,NaN,1934,NaN,NaN,NaN,Русский,NaN,NaN,Классическая литература,Н. Кравцовстр. 5-645,3,1,17,1,0,Academia,NaN
4,Собрание сочинений в двадцати шести томах. Том...,Эмиль Золя,NaN,1964,664,84x108/32,твёрдая,Русский,297000,NaN,"Зарубежная классика, Классическая проза, Литер...","М. Роммстр. 7Человек-зверь — Эмиль Золя, рома...","4,1",24,17,95,236,Художественная литература,NaN
13,Явившийся в пламени,Тэд Уильямс,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Зарубежное фэнтези,Н. Виленская,"3,8",82,85,4,0,Остен Ард,NaN
14,Железо внутри,Роб Сандерс,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Боевая фантастика, Зарубежная фантастика, Расс...",Е. Ластовцева,"4,7",15,8,1,0,"Ересь Хоруса, книга №16.8",NaN
21,Аптечная ведьма,Наталья Колесова,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"4,2",113,62,2,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31024,Табачная дорога,Эрскин Колдуэлл,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Зарубежная литература, Литература 20 века, Зар...",Лидия Слонимская,"4,5",6,135,1,0,NaN,NaN
31050,Новый год в октябре,Андрей Молчанов,NaN,1984,272,84х108/32,Мягкая,Русский,30000,NaN,Современная проза,NaN,"3,9",9,9,2,14,Советский писатель,NaN
31058,Когда я был вожатым,Николай Богданов,NaN,1986,208,NaN,NaN,Русский,NaN,12,Детская познавательная и развивающая литератур...,NaN,"4,2",60,112,10,2,Детская литература,NaN
31084,Бремя человека,Роберт Шекли,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Зарубежная фантастика, Юмористическая фантасти...",Ревекка Гальперина,4,64,10,1,1,NaN,NaN


In [ ]:
df_isbn_nans.isnull().sum()

BookTitle             0
Author               29
ISBN               4803
EditionYear        2031
Pages              3280
Size               3753
CoverType          3606
Language           2117
CopiesIssued       3700
AgeRestrictions    4141
Genres              937
TranslatorName     2768
Rating                0
HaveRead              0
Planned               0
Reviews               0
Quotes                0
Series             1495
Edition            3653
dtype: int64

As we see, among these 5176 books a lot of None values. This means we can safely remove this rows.

In [ ]:
df_grouped.isnull().sum()

ISBN                   0
BookTitle              0
Author               135
EditionYear           35
Pages               6255
Size               13424
CoverType           9810
Language             333
CopiesIssued       13606
AgeRestrictions    11348
Genres               740
TranslatorName     13434
Rating                 0
HaveRead               0
Planned                0
Reviews                0
Quotes                 0
Series               137
Edition             4254
dtype: int64

# Entries with None values

Remove books with more than 5 None cells, since inplacing this much of data will require too much effort and will not pay out. 

In [ ]:
df_filtered = df_grouped[df_grouped.isnull().sum(axis=1) < 5]
df_filtered.isnull().sum()

ISBN                  0
BookTitle             0
Author              100
EditionYear           4
Pages              1415
Size               8081
CoverType          4542
Language             54
CopiesIssued       8155
AgeRestrictions    7365
Genres               61
TranslatorName     8914
Rating                0
HaveRead              0
Planned               0
Reviews               0
Quotes                0
Series                2
Edition            2023
dtype: int64

# How much have been removed?

In [ ]:
df_grouped.isnull().sum() - df_filtered.isnull().sum()

ISBN                  0
BookTitle             0
Author               35
EditionYear          31
Pages              4840
Size               5343
CoverType          5268
Language            279
CopiesIssued       5451
AgeRestrictions    3983
Genres              679
TranslatorName     4520
Rating                0
HaveRead              0
Planned               0
Reviews               0
Quotes                0
Series              135
Edition            2231
dtype: int64

In [ ]:
df_grouped.shape[0] - df_filtered.shape[0]

5544

In [ ]:
df_filtered = df_filtered.replace(to_replace='None', value=np.nan)
df_filtered.sample(20)

,ISBN,BookTitle,Author,EditionYear,Pages,Size,CoverType,Language,CopiesIssued,AgeRestrictions,Genres,TranslatorName,Rating,HaveRead,Planned,Reviews,Quotes,Series,Edition
15201,978-5-389-04529-3,Его верный убийца. Книга 1. Жестокое милосердие,Робин Ла Фиверс,2012,480,None,Твердый,Русский,6000,16,"Зарубежное фэнтези, Историческое фэнтези",Мария Семенова,4,477,1026,42,33,Lady Fantasy,Азбука
10649,978-5-17-087193-3,Триумфальная арка,Эрих Мария Ремарк,2015,512,None,Твердый,Русский,7500,16,"Зарубежная классика, Классическая проза, Литер...",Михаил Рудницкий,"4,5",46318,23434,1074,6247,Возвращение с Западного фронта,АСТ
15618,978-5-389-07450-7,Один плюс один,Джоджо Мойес,2014,480,None,Твердый,Русский,30000,16,"Зарубежные любовные романы, Современные любовн...",А. Киланова,"4,3",5932,3895,318,244,Джоджо Мойес,"Азбука-Аттикус, Иностранка"
1144,5-17-012596-8,Его грешные пути,Саманта Джеймс,2002,320,None,Мягкая,Русский,None,None,Зарубежные любовные романы,None,"3,7",35,44,2,0,Шарм. Коллекция,АСТ
24166,978-5-9922-0442-1,Граф Монте-Кристо,Александр Дюма,2009,1248,None,None,Русский,None,16,"Исторические приключения, Классическая литерат...","В. Строев, Л. Олавская","4,7",35547,26655,497,1563,Полное издание в одном томе,Альфа-книга
21728,978-5-86218-501-0,Старый английский барон,Клара Рив,2012,270,70x90/16,твёрдая,Русский,1200,None,"Зарубежная классика, Мистика, Литература 18 века",None,4,37,150,4,0,Литературные памятники,"Наука, Ладомир"
23018,978-5-91878-045-9,Сожжение Просперо,Дэн Абнетт,2012,384,84х108/32,твердый,Русский,None,None,"Зарубежное фэнтези, Боевая фантастика",И. Савельева,"4,3",322,87,7,9,The Horus Heresy,Фантастика Книжный Клуб
1856,5-212-00330-X,Радуница,Сергей Есенин,1990,78,60x84/16 (145х200 мм),Мягкая,Русский,100000,None,Стихи и поэзия,None,4,3,2,0,0,"Библиотека репринтных изданий ""Книжные редкости""",Kniga
7550,978-5-00100-183-6,"Как понять акварель. Руководство для тех, кто ...",Том Хоффманн,2016,208,70x100/8,твёрдая,Русский,None,12,Изобразительное искусство,Анна Авдеева,"4,2",120,275,11,44,МИФ. Творчество,"Манн, Иванов и Фербер"
15606,978-5-389-07361-6,1900-й. Легенда о пианисте,Алессандро Барикко,2014,128,75x100/32 (115x177 мм),Мягкая,Русский,4000,16,Современная зарубежная литература,Наталия Колесовастр. 15-84Любитель сложностиАв...,"4,2",3165,3389,321,245,Азбука-классика (pocket-book) — Классика XX века,Азбука


# None values in EditionYear column.

Only 4 values have None. Manually google and inplace data.

In [ ]:
df_filtered.loc[df_filtered['EditionYear'].isnull()]

,ISBN,BookTitle,Author,EditionYear,Pages,Size,CoverType,Language,CopiesIssued,AgeRestrictions,Genres,TranslatorName,Rating,HaveRead,Planned,Reviews,Quotes,Series,Edition
5036,5-87365-060-8,Коготь дракона,Роберт Говард,None,432,84x108/32 (130x200 мм),твёрдая,Русский,5000,None,Зарубежное фэнтези,None,"3,2",4,8,0,0,Fantasy,Северо-Запад
7752,978-5-04-091574-3,Делириум,Лорен Оливер,None,464,75x108/32 (130x184 мм),твёрдая,Русский,None,None,"Зарубежная фантастика, Социальная фантастика, ...",None,4,9963,5070,398,994,Дистопия-Ультра. Фантастика от Лорен Оливер,Like Book
15461,978-5-389-06418-8,Молли Мун останавливает время,Джорджия Бинг,None,400,140х210,None,Русский,5,12,"Зарубежные детские книги, Детская фантастика",None,"4,3",818,124,12,1,Молли Мун,Азбука
17075,978-5-4467-1296-0,Собака на сене,Лопе де Вега,None,230,None,мягкая,Русский,None,None,"Зарубежная драматургия, Зарубежная классика, П...",Михаил Лозинский,"4,2",478,259,23,61,Библиотека драматургии Агентства ФТМ,ФТМ


In [ ]:
df_filtered.at[5036, 'EditionYear'] = 1999
df_filtered.at[7752, 'EditionYear'] = 2018
df_filtered.at[15461, 'EditionYear'] = 2014
df_filtered.at[17075, 'EditionYear'] = 1938

df_filtered.isnull().sum()

ISBN                  0
BookTitle             0
Author              100
EditionYear           0
Pages              1415
Size               8081
CoverType          4542
Language             54
CopiesIssued       8155
AgeRestrictions    7365
Genres               61
TranslatorName     8914
Rating                0
HaveRead              0
Planned               0
Reviews               0
Quotes                0
Series                2
Edition            2023
dtype: int64

# None Genres 

Genres are hard to interpolate, so remove 61 according rows.

In [ ]:
df_filtered = df_filtered[df_filtered['Genres'].notna()]
df_filtered.isnull().sum()

ISBN                  0
BookTitle             0
Author              100
EditionYear           0
Pages              1411
Size               8061
CoverType          4540
Language             54
CopiesIssued       8126
AgeRestrictions    7313
Genres                0
TranslatorName     8895
Rating                0
HaveRead              0
Planned               0
Reviews               0
Quotes                0
Series                2
Edition            2013
dtype: int64

# None Series

Only 2 rows have no Series values. It's would be easy to implace data.

In [ ]:
df_filtered.at[3026, 'Series'] = 'Только для маленьких колдуний!'
df_filtered.at[3026, 'Edition'] = 'Оникс'

df_filtered.at[5447, 'Series'] = 'Отдельное издание'
df_filtered.at[5447, 'Edition'] = 'Детектив-Пресс'

df_filtered.loc[df_filtered['Series'].isnull()]

,ISBN,BookTitle,Author,EditionYear,Pages,Size,CoverType,Language,CopiesIssued,AgeRestrictions,Genres,TranslatorName,Rating,HaveRead,Planned,Reviews,Quotes,Series,Edition


# Size column data format

Size column have size of cover and size of paper. These values are interchangeable. Let's separate values into two columns and then use to reduce None values.

In [ ]:
df_filtered['Size'].value_counts()

84x108/32 (130х205 мм)         1443
84x108/32 (130x200 мм)         1306
84x108/32 (130х200 мм)          827
76x100/32 (115x180 мм)          803
84x108/32 (~130х205 мм)         533
                               ... 
60 x90/16 (~145х217 мм)           1
22 x 14.5 x 2.8                   1
84х108/32 (126х200)               1
130x250                           1
130х165 мм (средний формат)       1
Name: Size, Length: 968, dtype: int64

In [ ]:
df_filtered['Size'] = df_filtered['Size'].str.replace(r'[XxХх×х*]', 'x', regex=True).astype('str')
df_filtered['Size'] = df_filtered['Size'].str.replace(r' [\\\/]', '/', regex=True).astype('str')

df_filtered['Size'] = df_filtered['Size'].str.replace(r' ', '', regex=True).astype('str')

df_filtered['Size'] = df_filtered['Size'].str.replace(r'.', '', regex=True).astype('str')
df_filtered['Size'] = df_filtered['Size'].str.replace(r',', '', regex=True).astype('str')
df_filtered['Size'] = df_filtered['Size'].str.replace(r'~', '', regex=True).astype('str')
df_filtered['Size'] = df_filtered['Size'].str.replace(r';', '', regex=True).astype('str')
df_filtered['Size'] = df_filtered['Size'].str.replace(r'см', '0', regex=True).astype('str')
df_filtered['Size'] = df_filtered['Size'].str.replace(r'cm', '0', regex=True).astype('str')
df_filtered['Size'] = df_filtered['Size'].str.replace(r'мм', '', regex=True).astype('str')
df_filtered['Size'] = df_filtered['Size'].str.replace(r'mm', '', regex=True).astype('str')


df_filtered['PaperSize'] = df_filtered['Size'].str.extract(r'(\d+[хХxX]\d+[\\\/]\d+)', expand=True)

df_filtered['Size'] = df_filtered['Size'].str.replace(r'(\d+[XxХх]\d+[\\\/]\d+)', "")
df_filtered['Size'] = df_filtered['Size'].str.replace(r'.+"', "")
df_filtered['Size'] = df_filtered['Size'].str.replace(r'.*:', "")


df_filtered['Size'] = df_filtered['Size'].str.replace(r'(', '', regex=True).astype('str')
df_filtered['Size'] = df_filtered['Size'].str.replace(r')', '', regex=True).astype('str')

df_filtered['Size'] = df_filtered['Size'].str.replace('Тираж.+', '').astype('str')

df_filtered['Size'] = df_filtered['Size'].str.replace('[3-9][0-9]{2}[а-яіА-Я]+', '').astype('str')
df_filtered['Size'] = df_filtered['Size'].str.replace('[а-яіА-Я]+', '').astype('str')
df_filtered['Size'] = df_filtered['Size'].str.replace('[а-яіёА-Я]+', '').astype('str')

df_filtered['Size'] = df_filtered['Size'].str.replace('&nbsp', '').astype('str')

df_filtered['Size'] = df_filtered['Size'].replace(r'^\s*$', np.nan, regex=True).astype('str')
df_filtered['Size'] = df_filtered['Size'].replace(r'[.]', np.nan, regex=True).astype('str')
df_filtered['Size'] = df_filtered['Size'].replace(r'^\/', '', regex=True).astype('str')
df_filtered['Size'] = df_filtered['Size'].replace(r'None', np.nan, regex=True).astype('str')




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykern

,ISBN,BookTitle,Author,EditionYear,Pages,Size,CoverType,Language,CopiesIssued,AgeRestrictions,Genres,TranslatorName,Rating,HaveRead,Planned,Reviews,Quotes,Series,Edition,PaperSize
0,0-000-00000-0,Опоздавшие к лету: Колдун. Мост Ватерлоо. Аттр...,Андрей Лазарчук,1990,276,109x177,твёрдая,Русский,45000,16,Отечественная фантастика,None,4,81,132,18,44,"Новая фантастика, вып.IV",Латвийский детский фонд,76x90/32
6,0-15-637609-1,Почетный караул,Джеймс Гулд Коззенс,1992,640,130x200,Твердый,Русский,25000,None,Современная проза,"А. Балясников, И. Шевченко, Владимир Баканов",4,1,33,0,0,Радуга,None,84x108/32
15,0-316-76948-7,The Catcher in the Rye,J. D. Salinger,1991,224,105x170,Мягкая,Английский,None,16,"Зарубежная классика, Классическая проза, Реализм",None,"3,7",77836,17159,1573,2556,Back Bay Books,None,NaN
17,0-330-48022-7,Misspent Youth,Питер Гамильтон,2003,448,nan,Мягкая,Русский,None,None,Зарубежная фантастика,None,0,9,0,0,0,"Вселенная Содружества, книга №1",Pan Books,NaN
26,0-399-13615-0,Все страхи мира,Том Клэнси,1993,767,220x240,Твердый,Русский,100000,None,"Триллеры, Политические детективы, Шпионские де...",И.Почиталинстр. 7-762,4,52,55,2,0,Зарубежный триллер,Мир,84x104/32


In [ ]:
df_filtered.loc[df_filtered['Size']=='76x108x32','PaperSize']='76x108/32'
df_filtered.loc[df_filtered['Size']=='76x108x32','Size']='240x330'
df_filtered.loc[df_filtered['Size']=='84x108x32','PaperSize']='84x108/32'
df_filtered.loc[df_filtered['Size']=='84x108x32','Size']='130x200'
df_filtered.loc[df_filtered['Size']=='60×88/16','PaperSize']='60×88/16'
df_filtered.loc[df_filtered['Size']=='60×88/16','Size']='145x205'
df_filtered.loc[df_filtered['Size']=='84×108/32','PaperSize']='84×108/32'
df_filtered.loc[df_filtered['Size']=='84×108/32','Size']='130x200'
df_filtered.loc[df_filtered['Size']=='84/108/32','PaperSize']='84x108/32'
df_filtered.loc[df_filtered['Size']=='84/108/32','Size']='130x200'


df_filtered.loc[df_filtered['Size']=='13x210','Size']='130x210'
df_filtered.loc[df_filtered['Size']=='233x310x10','Size']='233x310'
df_filtered.loc[df_filtered['Size']=='115x18','Size']='115x180'
df_filtered.loc[df_filtered['Size']=='125x2050','Size']='125x205'
df_filtered.loc[df_filtered['Size']=='207x133x28','Size']='133x207'
df_filtered.loc[df_filtered['Size']=='107x1650','Size']='107x165'
df_filtered.loc[df_filtered['Size']=='218x145x4','Size']='145x218'
df_filtered.loc[df_filtered['Size']=='206x131x28','Size']='131x206'
df_filtered.loc[df_filtered['Size']=='112x1650','Size']='131x206'
df_filtered.loc[df_filtered['Size']=='206x13x26','Size']='130x206'
df_filtered.loc[df_filtered['Size']=='104x1650','Size']='104x165'
df_filtered.loc[df_filtered['Size']=='205x150x30','Size']='150x205'
df_filtered.loc[df_filtered['Size']=='116x1900','Size']='116x190'
df_filtered.loc[df_filtered['Size']=='166x112x17','Size']='112x166'
df_filtered.loc[df_filtered['Size']=='230x','Size']='230x290'
df_filtered.loc[df_filtered['Size']=='16500x10500x1900','Size']='105x165'
df_filtered.loc[df_filtered['Size']=='205x13x2','Size']='130x205'
df_filtered.loc[df_filtered['Size']=='12x18','Size']='120x180'
df_filtered.loc[df_filtered['Size']=='13x21','Size']='130x210'
df_filtered.loc[df_filtered['Size']=='13x200','Size']='130x200'
df_filtered.loc[df_filtered['Size']=='125x1770','Size']='125x177'
df_filtered.loc[df_filtered['Size']=='125x20','Size']='125x200'
df_filtered.loc[df_filtered['Size']=='17x24','Size']='170x240'
df_filtered.loc[df_filtered['Size']=='13x185','Size']='130x185'
df_filtered.loc[df_filtered['Size']=='11x170','Size']='110x170'
df_filtered.loc[df_filtered['Size']=='171x2730','Size']='171x273'
df_filtered.loc[df_filtered['Size']=='129x1980','Size']='129x198'
df_filtered.loc[df_filtered['Size']=='2050x1350','Size']='135x205'
df_filtered.loc[df_filtered['Size']=='15x21','Size']='150x210'
df_filtered.loc[df_filtered['Size']=='12x17','Size']='120x170'
df_filtered.loc[df_filtered['Size']=='15x22','Size']='150x220'


df_filtered.head()

In [ ]:
df_filtered.loc[df_filtered['PaperSize']=='84x108/32', 'Size'] = '130x200'
df_filtered.loc[df_filtered['PaperSize']=='60x90/16', 'Size'] = '145x215'
df_filtered.loc[df_filtered['PaperSize']=='76x100/32', 'Size'] = '130х165'
df_filtered.loc[df_filtered['PaperSize']=='70x90/32', 'Size'] = '107x165'
df_filtered.loc[df_filtered['PaperSize']=='70x100/32', 'Size'] = '120x165'
df_filtered.loc[df_filtered['PaperSize']=='70x100/16', 'Size'] = '170x240'
df_filtered.loc[df_filtered['PaperSize']=='75x100/32', 'Size'] = '120x177'
df_filtered.loc[df_filtered['PaperSize']=='60x84/16', 'Size'] = '145x200'
df_filtered.loc[df_filtered['PaperSize']=='80x100/32', 'Size'] = '120x190'
df_filtered.loc[df_filtered['PaperSize']=='70x90/16', 'Size'] = '170x215'
df_filtered.loc[df_filtered['PaperSize']=='70x108/32', 'Size'] = '130x165'
df_filtered.loc[df_filtered['PaperSize']=='84x1081/32', 'Size'] = '130x200'
df_filtered.loc[df_filtered['PaperSize']=='60x100/16', 'Size'] = '145x240'
df_filtered.loc[df_filtered['PaperSize']=='84x108/16', 'Size'] = '205x260'
df_filtered.loc[df_filtered['PaperSize']=='60x901/16', 'Size'] = '130x165'
df_filtered.loc[df_filtered['PaperSize']=='84x108\32', 'Size'] = '130x200'
df_filtered.loc[df_filtered['PaperSize']=='60x90/8', 'Size'] = '205x290'
df_filtered.loc[df_filtered['PaperSize']=='70x108/16', 'Size'] = '170x260'
df_filtered.loc[df_filtered['PaperSize']=='84x100/32', 'Size'] = '120x200'
df_filtered.loc[df_filtered['PaperSize']=='76x108/32', 'Size'] = '135x205'

df_filtered.loc[df_filtered['Size']=='130x200', 'PaperSize'] = '84x108/32'
df_filtered.loc[df_filtered['Size']=='145x215', 'PaperSize'] = '60x90/16'
df_filtered.loc[df_filtered['Size']=='130х165', 'PaperSize'] = '76x100/32'
df_filtered.loc[df_filtered['Size']=='107x165', 'PaperSize'] = '70x90/32'
df_filtered.loc[df_filtered['Size']=='120x165', 'PaperSize'] = '70x100/32'
df_filtered.loc[df_filtered['Size']=='170x240', 'PaperSize'] = '70x100/16'
df_filtered.loc[df_filtered['Size']=='120x177', 'PaperSize'] = '75x100/32'
df_filtered.loc[df_filtered['Size']=='145x200', 'PaperSize'] = '60x84/16'
df_filtered.loc[df_filtered['Size']=='120x190', 'PaperSize'] = '80x100/32'
df_filtered.loc[df_filtered['Size']=='170x215', 'PaperSize'] = '70x90/16'
df_filtered.loc[df_filtered['Size']=='130x165', 'PaperSize'] = '70x108/32'

df_filtered['Size'] = df_filtered['Size'].str.replace(r'[XxХх×х*]', 'x', regex=True).astype('str')

In [ ]:
pd.set_option("display.max_rows", None)
df_filtered['Size'].value_counts()

nan                  8186
130x200              5360
145x215              1470
130x165              1236
107x165               513
120x165               349
115x180               240
170x240               238
120x177               202
145x200               198
135x205               191
120x190               166
170x215               159
125x200               155
140x210               115
205x260                84
220x240                81
170x260                69
120x200                69
126x200                63
145x240                61
205x290                60
205x240                37
150x210                32
130x184                29
115x200                28
138x212                26
116x190                23
107x177                22
130x195                22
120x205                21
110x180                21
170x230                20
125x190                20
130x205                16
130x190                16
112x184                15
265x340                13
140x200     

In [ ]:
df_filtered.shape

(20308, 20)

# Filter not matching rows

After separating the data in Size column, we need to filter data that are left and not fit to be left in table. Leave only number x number or nan value.

In [ ]:
filter_wrong_style = df_filtered['Size'].str.contains('^(\d+x\d+){1}$')
filter_nan = df_filtered['Size'].str.contains('nan')
filter_total = filter_nan + filter_wrong_style 
filter_total.value_counts()
df_unf = df_filtered[filter_total]
df_unf.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,ISBN,BookTitle,Author,EditionYear,Pages,Size,CoverType,Language,CopiesIssued,AgeRestrictions,Genres,TranslatorName,Rating,HaveRead,Planned,Reviews,Quotes,Series,Edition,PaperSize
0,0-000-00000-0,Опоздавшие к лету: Колдун. Мост Ватерлоо. Аттр...,Андрей Лазарчук,1990,276,109x177,твёрдая,Русский,45000,16,Отечественная фантастика,None,4,81,132,18,44,"Новая фантастика, вып.IV",Латвийский детский фонд,76x90/32
6,0-15-637609-1,Почетный караул,Джеймс Гулд Коззенс,1992,640,130x200,Твердый,Русский,25000,None,Современная проза,"А. Балясников, И. Шевченко, Владимир Баканов",4,1,33,0,0,Радуга,None,84x108/32
15,0-316-76948-7,The Catcher in the Rye,J. D. Salinger,1991,224,105x170,Мягкая,Английский,None,16,"Зарубежная классика, Классическая проза, Реализм",None,"3,7",77836,17159,1573,2556,Back Bay Books,None,NaN
17,0-330-48022-7,Misspent Youth,Питер Гамильтон,2003,448,nan,Мягкая,Русский,None,None,Зарубежная фантастика,None,0,9,0,0,0,"Вселенная Содружества, книга №1",Pan Books,NaN
26,0-399-13615-0,Все страхи мира,Том Клэнси,1993,767,220x240,Твердый,Русский,100000,None,"Триллеры, Политические детективы, Шпионские де...",И.Почиталинстр. 7-762,4,52,55,2,0,Зарубежный триллер,Мир,84x104/32


In [ ]:
df_unf.shape

(20189, 20)

In [ ]:
df_unf['Size'].value_counts()

nan         8186
130x200     5360
145x215     1470
130x165     1236
107x165      513
120x165      349
115x180      240
170x240      238
120x177      202
145x200      198
135x205      191
120x190      166
170x215      159
125x200      155
140x210      115
205x260       84
220x240       81
120x200       69
170x260       69
126x200       63
145x240       61
205x290       60
205x240       37
150x210       32
130x184       29
115x200       28
138x212       26
116x190       23
130x195       22
107x177       22
110x180       21
120x205       21
170x230       20
125x190       20
130x205       16
130x190       16
112x184       15
265x340       13
140x200       13
110x175       12
125x195       12
105x165       11
110x165       11
210x210       10
245x340        9
130x210        9
105x177        9
180x115        9
100x140        9
140x205        9
120x195        8
170x205        8
135x210        8
107x140        8
145x210        8
112x165        8
145x220        7
120x180        7
107x200       

In [ ]:
df_unf.loc[df_unf['Size']=='nan', 'Size'] = None
df_unf.isnull().sum()

# Convert CoverType to one format 

There are 4 values that represent 2 types of cover. Convert them to 0 and 1.

In [ ]:
df_unf.loc[df_unf['CoverType'].str.startswith('Т', na=False), 'CoverType'] = 1
df_unf.loc[df_unf['CoverType'].str.startswith('т', na=False), 'CoverType'] = 1
df_unf.loc[df_unf['CoverType'].str.startswith('М', na=False), 'CoverType'] = 0
df_unf.loc[df_unf['CoverType'].str.startswith('м', na=False), 'CoverType'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
df_unf.shape

(20189, 20)

# Further remove null data

Remove rows which contain None in Author and Language features.



In [ ]:
df = df_unf[~df_unf['Author'].isnull()]
df = df[~df['Language'].isnull()]
df.isnull().sum()

ISBN                  0
BookTitle             0
Author                0
EditionYear           0
Pages              1370
Size               8145
CoverType          4493
Language              0
CopiesIssued       7985
AgeRestrictions    7202
Genres                0
TranslatorName     8785
Rating                0
HaveRead              0
Planned               0
Reviews               0
Quotes                0
Series                0
Edition            1980
PaperSize          9314
dtype: int64

In [ ]:
df.isnull().sum()

ISBN                  0
BookTitle             0
Author                0
EditionYear           0
Pages              1370
Size               8145
CoverType          4493
Language              0
CopiesIssued       7985
AgeRestrictions    7202
Genres                0
TranslatorName     8785
Rating                0
HaveRead              0
Planned               0
Reviews               0
Quotes                0
Series                0
Edition            1980
PaperSize          9314
dtype: int64

# Inspect Genres which have no value in AgeRestrictions feature



In [ ]:
df.loc[df['AgeRestrictions'].isnull(), 'Genres'].value_counts()

Современная зарубежная литература                                                                                             528
Зарубежная классика                                                                                                           272
Зарубежная фантастика                                                                                                         208
Современная проза                                                                                                             201
Современная зарубежная литература, Зарубежные детективы                                                                       163
Биографии и мемуары                                                                                                           158
Зарубежные любовные романы                                                                                                    143
Зарубежное фэнтези                                                                        

# Google age restrictions for each Genre and manually enter value

In [ ]:
df.loc[(df['Genres'].str.contains('манга') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 11
df.loc[(df['Genres'].str.contains('Детские детективы') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 8
df.loc[(df['Genres'].str.contains('Детские книги') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 7
df.loc[(df['Genres'].str.contains('Сказки') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 6
df.loc[(df['Genres'].str.contains('Триллеры') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 13
df.loc[(df['Genres'].str.contains('Ужасы') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 13
df.loc[(df['Genres'].str.contains('Книги для подростков') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12
df.loc[(df['Genres'].str.contains('Детская проза') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 8
df.loc[(df['Genres'].str.contains('Русская литература для детей') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 8
df.loc[(df['Genres'].str.contains('Любовные романы') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 16

df.loc[(df['Genres'].str.contains('Детская фантастика') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 8
df.loc[(df['Genres'].str.contains('Зарубежные любовные романы') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 16


df.loc[(df['Genres'].str.contains('Фэнтези') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 13
df.loc[(df['Genres'].str.contains('фэнтези') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 13

df.loc[(df['Genres'].str.contains('Зарубежные детективы') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 13
df.loc[(df['Genres'].str.contains('Детективы') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 13
df.loc[(df['Genres'].str.contains('Стихи и поэзия') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 11
df.loc[(df['Genres'].str.contains('Биографии') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 13


df.loc[(df['Genres'].str.contains('Зарубежные детские книги') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 8
df.loc[(df['Genres'].str.contains('Школьные методические пособия') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 10
df.loc[(df['Genres'].str.contains('Военное дело') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 14
df.loc[(df['Genres'].str.contains('Книги о войне') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 14
df.loc[(df['Genres'].str.contains('Юмор и сатира') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12

df.loc[(df['Genres'].str.contains('Зарубежная') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12
df.loc[(df['Genres'].str.contains('Зарубежное') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12
df.loc[(df['Genres'].str.contains('зарубежная') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12
df.loc[(df['Genres'].str.contains('зарубежное') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12

df.loc[(df['Genres'].str.contains('классика') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12
df.loc[(df['Genres'].str.contains('Отечественные') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12
df.loc[(df['Genres'].str.contains('детектив') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 13
df.loc[(df['Genres'].str.contains('Отечественная') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12
df.loc[(df['Genres'].str.contains('Русское') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12
df.loc[(df['Genres'].str.contains('фантастика') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 13
df.loc[(df['Genres'].str.contains('Отечественные') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12
df.loc[(df['Genres'].str.contains('Отечественный') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12


df.loc[(df['Genres'].str.contains('Классическая') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12
df.loc[(df['Genres'].str.contains('риключения') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12
df.loc[(df['Genres'].str.contains('литература') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12

df.loc[(df['Genres'].str.contains('века') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12

df.loc[(df['Genres'].str.contains('Современная') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 12
df.loc[(df['Genres'].str.contains('История') & df['AgeRestrictions'].isnull()), 'AgeRestrictions'] = 11


df.loc[df['AgeRestrictions'].isnull(), 'AgeRestrictions'] = 14

df['AgeRestrictions'] = df['AgeRestrictions'].astype('int64')

In [ ]:
df['AgeRestrictions'].value_counts()

16    7900
12    6282
13    2060
18    1715
6      767
14     429
11     330
8      295
7      181
9       27
10      24
3       12
5        5
4        4
2        3
15       2
1        1
Name: AgeRestrictions, dtype: int64

In [ ]:
df.loc[df['AgeRestrictions'].isnull(), 'Genres'].value_counts()

Series([], Name: Genres, dtype: int64)

# Transliterate Genres

Turn Genres into separate columns:

1) Download transliterate module

2) Split genres and make one general vector

3) Iterate through rows, split genres of each book and place 1 if one have according genre 

In [ ]:
!pip install transliterate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 4.0 MB/s 


In [ ]:
from transliterate import translit

df_2 = df

orig_columns = df.columns

genres = df['Genres'].values
genres = set(translit((','.join(genres)
                          .replace(', ', ',')
                          .replace(' ', '_')
                          .lower()
                          .strip()),"ru", reversed=True).split(','))
df[list(genres)]=0

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3645: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = value


In [ ]:
for index, row in df.iterrows():
  genres_row = set(translit(row['Genres'], "ru", reversed=True)
                          .replace(', ', ',')
                          .replace(' ', '_')
                          .split(','))
  for genre in genres_row:
    df.at[index, genre.lower()] = 1


# With separage genres, interpolate numbers of pages where necessary

In [ ]:
df_inter = df

df_inter['Pages'] = df_inter['Pages'].astype('float32')
df_inter['Pages'].interpolate(method='linear', inplace=True, limit_direction="both", axis=0)

In [ ]:
df_inter.loc[df_inter['CopiesIssued'].isnull()].isnull().sum()

ISBN                                                        0
BookTitle                                                   0
Author                                                      0
EditionYear                                                 0
Pages                                                       0
Size                                                     5258
CoverType                                                2644
Language                                                    0
CopiesIssued                                             7985
AgeRestrictions                                             0
Genres                                                      0
TranslatorName                                           3424
Rating                                                      0
HaveRead                                                    0
Planned                                                     0
Reviews                                                     0
Quotes  

# Drop features with many None values, and wich would be impossible to interpolate

In [ ]:
df_dropped = df_inter.drop(columns=['TranslatorName', 'PaperSize'], axis=1)
df_dropped = df_dropped.drop(df_dropped.loc[df_dropped['CopiesIssued'].isnull()].index)
df_dropped.isnull().sum()

ISBN                                                        0
BookTitle                                                   0
Author                                                      0
EditionYear                                                 0
Pages                                                       0
Size                                                     2887
CoverType                                                1849
Language                                                    0
CopiesIssued                                                0
AgeRestrictions                                             0
Genres                                                      0
Rating                                                      0
HaveRead                                                    0
Planned                                                     0
Reviews                                                     0
Quotes                                                      0
Series  

In [ ]:
df_dropped.shape

(12052, 386)

In [ ]:
df_dropped['Pages'] = df_dropped['Pages'].astype('int32')

In [ ]:
df_dropped['Size'].value_counts()

130x200     4410
145x215     1128
130x165     1017
107x165      458
120x165      305
120x177      188
170x240      187
145x200      163
170x215      147
120x190      133
135x205       95
220x240       77
205x260       68
120x200       64
205x290       59
125x200       53
145x240       35
115x180       34
205x240       34
170x260       33
130x184       26
150x210       25
140x210       23
107x177       22
120x205       21
170x230       19
112x184       15
130x190       13
265x340       13
130x205       12
126x200       11
125x190       10
170x205        8
107x140        7
120x195        7
245x340        7
210x210        7
107x200        7
116x190        7
140x200        6
110x180        6
135x210        6
180x115        6
140x205        6
105x165        6
145x220        5
100x140        5
210x280        5
130x210        5
90x130         4
128x182        4
138x212        4
233x310        4
109x177        4
88x108         4
125x205        4
270x275        3
120x185        3
138x200       

In [ ]:
df_dropped.loc[df_dropped['Size']=='143x2220', 'Size'] = '143x222'
df_dropped.loc[df_dropped['Size']=='138x2120', 'Size'] = '138x212'

# Split Size feature

Size feature contains Width (generraly first value) and Height (generally second value) of book. Split it into two features.

In [ ]:
df_dropped[['CoverWidth', 'CoverHeight']] = df_dropped['Size'].str.split('x', expand=True)

ser_size = df_dropped['Size']

In [ ]:
df_dropped = df_dropped.drop(columns=['Size'], axis=1)

In [ ]:
df_dropped.sample(5)

,ISBN,BookTitle,Author,EditionYear,Pages,CoverType,Language,CopiesIssued,AgeRestrictions,Genres,...,bibliografija,os_i_seti,_rasskazy,zarubezhnyj_jumor,komiksy,zarubezhnaja_poezija,klassiki_psihologii,tehnicheskie_nauki,CoverWidth,CoverHeight
2111,5-244-00364-X,Ловец орлов и другие повести (сборник),Джеймс Уиллард Шульц,1991,748,None,Русский,200000,12,Исторические приключения,...,0,0,0,0,0,0,0,0,130,200
22383,978-5-906640-11-6,Петсон идет в поход,Свен Нурдквист,2014,24,1,Русский,3000,6,"Зарубежные детские книги, Сказки",...,0,0,0,0,0,0,0,0,205,290
807,5-05-002641-4,Проза разных лет (сборник),Хорхе Луис Борхес,1989,320,1,Русский,50000,12,Современная проза,...,0,0,0,0,0,0,0,0,140,205
5977,5-94278011-0,Плотина против Тихого океана,Маргерит Дюрас,2000,280,1,Русский,5000,12,Современная зарубежная литература,...,0,0,0,0,0,0,0,0,None,None
22922,978-5-91759-226-8,Кот-убийца и Рождество,Энн Файн,2013,64,1,Русский,5000,12,"Зарубежные детские книги, Сказки",...,0,0,0,0,0,0,0,0,145,200


In [ ]:
df_dropped.loc[df_dropped['CoverType'].isnull(), 'EditionYear'].value_counts()

2010    283
2009    216
2015    193
2008    180
2011    164
2014     88
2007     77
2012     69
2016     69
2013     67
2006     52
2017     43
2005     41
2004     30
2003     28
1993     26
1994     24
2000     23
2018     19
1991     17
2002     17
2019     16
2001     14
1992     14
1997     13
1998     11
1996      9
1999      8
1988      7
1995      7
2021      6
1990      6
2020      6
1989      5
2014      1
Name: EditionYear, dtype: int64

In [ ]:
df_dropped.shape

(12052, 387)

# Interpolate CoverType feature



In [ ]:
df_dropped['CoverType'] = df_dropped['CoverType'].astype('category')
df_dropped['CoverType'].dtype

CategoricalDtype(categories=[0, 1], ordered=False)

In [ ]:
df_dropped['CoverType'] = (df_dropped['CoverType'].cat.codes.replace(-1, np.nan)
                                  .interpolate().astype(int).astype('category')
                                  .cat.rename_categories(df_dropped['CoverType'].cat.categories))

In [ ]:
df_dropped['CoverType'].value_counts()

1    9138
0    2914
Name: CoverType, dtype: int64

In [ ]:
df_dropped.isnull().sum()

ISBN                                                        0
BookTitle                                                   0
Author                                                      0
EditionYear                                                 0
Pages                                                       0
CoverType                                                   0
Language                                                    0
CopiesIssued                                                0
AgeRestrictions                                             0
Genres                                                      0
Rating                                                      0
HaveRead                                                    0
Planned                                                     0
Reviews                                                     0
Quotes                                                      0
Series                                                      0
Edition 

# None in Edition feature

Sometimes Edition is not being written when book is published, so it has only one feature Series. 

Make Edition same as Series if Edition is None

In [ ]:
df_dropped.loc[df_dropped['Edition'].isnull()].sample(5)

,ISBN,BookTitle,Author,EditionYear,Pages,CoverType,Language,CopiesIssued,AgeRestrictions,Genres,...,bibliografija,os_i_seti,_rasskazy,zarubezhnyj_jumor,komiksy,zarubezhnaja_poezija,klassiki_psihologii,tehnicheskie_nauki,CoverWidth,CoverHeight
23984,978-5-9908778-7-0,Счастливы неимущие (Евангелие от Матфея). Суде...,Михаил Барщевский,2017,832,0,Русский,500,12,"Документальная литература, Статьи",...,0,0,0,0,0,0,0,0,145,215
22098,978-5-901682-77-7,Тропами ада,Людвиг Павельчик,2009,272,1,Русский,1000,16,"Ужасы, мистика",...,0,0,0,0,0,0,0,0,145,200
21784,978-5-86471-604-5,Дневник миссис Фрай,Эдна Фрай,2011,256,1,Русский,5000,13,"Биографии и мемуары, Современная зарубежная ли...",...,0,0,0,0,0,0,0,0,130,200
14117,978-5-367-00564-6,Голос бамбука (сборник),Ясунари Кавабата,2007,608,1,Русский,3000,12,"Зарубежная классика, Современная зарубежная ли...",...,0,0,0,0,0,0,0,0,145,215
21682,978-5-8370-0482-7,Сталки и компания,Редьярд Киплинг,2008,304,1,Русский,5000,12,"Детская проза, Зарубежные детские книги, Заруб...",...,0,0,0,0,0,0,0,0,130,200


In [ ]:
df_dropped['Edition'].fillna(df_dropped['Series'], inplace=True)
df_dropped.isnull().sum()

ISBN                                                        0
BookTitle                                                   0
Author                                                      0
EditionYear                                                 0
Pages                                                       0
CoverType                                                   0
Language                                                    0
CopiesIssued                                                0
AgeRestrictions                                             0
Genres                                                      0
Rating                                                      0
HaveRead                                                    0
Planned                                                     0
Reviews                                                     0
Quotes                                                      0
Series                                                      0
Edition 

In [ ]:
df_dropped.shape

(12052, 387)

In [ ]:
df_dropped.sample(5)

,ISBN,BookTitle,Author,EditionYear,Pages,CoverType,Language,CopiesIssued,AgeRestrictions,Genres,...,bibliografija,os_i_seti,_rasskazy,zarubezhnyj_jumor,komiksy,zarubezhnaja_poezija,klassiki_psihologii,tehnicheskie_nauki,CoverWidth,CoverHeight
25732,9785436605951,Пески. Потомки джиннов,Элвин Гамильтон,2019,480,0,Русский,5000,16,"Зарубежное фэнтези, Книги для подростков, Фэнтези",...,0,0,0,0,0,0,0,0,130,200
3247,5-699-07308-6,Уотт,Сэмюэл Беккет,2004,416,1,Русский,3000,12,Зарубежная классика,...,0,0,0,0,0,0,0,0,120,165
17290,978-5-6040082-3-2,Бойцовые рыбки,С.Э. Хинтон,2018,160,1,Русский,3000,16,"Современная проза, Современная зарубежная лите...",...,0,0,0,0,0,0,0,0,None,None
3017,5-486-00783-3,Одинокая птица,Киоко Мори,2006,320,1,Русский,15000,12,"Современная зарубежная литература, Книги для п...",...,0,0,0,0,0,0,0,0,107,165
21629,978-5-8159-1319-6,Алмазная колесница,Борис Акунин,2015,704,1,Русский,5000,16,"Отечественные детективы, Отечественный мужской...",...,0,0,0,0,0,0,0,0,130,200


# Interpolate Width and Height

In [ ]:
df_dropped['CoverWidth'] = df_dropped['CoverWidth'].astype('category')
df_dropped['CoverHeight'] = df_dropped['CoverHeight'].astype('category')

df_dropped['CoverWidth']= (df_dropped['CoverWidth'].cat.codes.replace(-1, np.nan)
                                  .interpolate().astype(int).astype('category')
                                  .cat.rename_categories(df_dropped['CoverWidth'].cat.categories))

df_dropped['CoverHeight']= (df_dropped['CoverHeight'].cat.codes.replace(-1, np.nan)
                                  .interpolate().astype(int).astype('category')
                                  .cat.rename_categories(df_dropped['CoverHeight'].cat.categories))

df_dropped['CoverWidth'] = df_dropped['CoverWidth'].astype('int64')
df_dropped['CoverHeight'] = df_dropped['CoverHeight'].astype('int64')

In [ ]:
df_dropped.isnull().sum()

ISBN                                                     0
BookTitle                                                0
Author                                                   0
EditionYear                                              0
Pages                                                    0
CoverType                                                0
Language                                                 0
CopiesIssued                                             0
AgeRestrictions                                          0
Genres                                                   0
Rating                                                   0
HaveRead                                                 0
Planned                                                  0
Reviews                                                  0
Quotes                                                   0
Series                                                   0
Edition                                                 

In [ ]:
df_dropped[['CoverWidth', 'CoverHeight']].sample(15)

,CoverWidth,CoverHeight
4007,120,165
19281,107,165
2451,130,200
23443,120,177
22060,130,200
6373,120,200
10885,130,200
10154,130,165
24505,130,200
18834,130,200


In [ ]:
df_dropped = df_dropped.reset_index()

In [ ]:
df_dropped.shape

(12052, 388)

In [ ]:
df_dropped.isnull().sum()

index                                                    0
ISBN                                                     0
BookTitle                                                0
Author                                                   0
EditionYear                                              0
Pages                                                    0
CoverType                                                0
Language                                                 0
CopiesIssued                                             0
AgeRestrictions                                          0
Genres                                                   0
Rating                                                   0
HaveRead                                                 0
Planned                                                  0
Reviews                                                  0
Quotes                                                   0
Series                                                  